In [11]:
import numpy as np
import pandas as pd

In [12]:
import scipy.io

In [13]:
data = scipy.io.loadmat('/Users/Sophie/Library/CloudStorage/OneDrive-Personnel/IP/Data Stream Processing/PySAD_to_River/data/arrhythmia.mat') 

In [14]:
X = data['X']
y = data['y']

In [37]:
y.mean()

0.14601769911504425

In [43]:
X.max(axis=0)

array([ 83. ,   1. , 780. , 176. , 188. , 524. , 509. , 381. , 205. ,
       169. ,  88. , 156. ,  88. ,  24. ,   0. , 100. ,   1. ,   1. ,
         1. ,   1. ,   1. ,   1. ,  76. , 132. ,  92. ,  36. ,  56. ,
        76. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,  92. , 116. ,
       132. ,  64. ,  44. ,  92. ,   1. ,   1. ,   1. ,   1. ,   1. ,
         1. , 136. ,  80. ,  80. ,  84. ,  32. ,  92. ,   1. ,   1. ,
         1. ,   1. ,   1. ,   1. ,  88. , 148. ,  92. ,  44. ,   0. ,
        96. ,   0. ,   1. ,   1. ,   1. ,   1. ,   1. ,  88. , 128. ,
       120. ,  44. ,  44. ,  80. ,   1. ,   1. ,   0. ,   1. ,   1. ,
         1. , 140. , 216. , 116. ,  80. ,  28. , 116. ,   1. ,   1. ,
         1. ,   1. ,   1. ,   1. , 132. , 216. , 108. ,  72. ,  32. ,
        96. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,  96. , 132. ,
       112. ,  76. ,  36. ,  88. ,   1. ,   1. ,   1. ,   1. ,   1. ,
         1. ,  88. ,  92. , 124. ,  24. ,  40. ,  60. ,   1. ,   1. ,
         0. ,   0. ,

In [44]:
X.min(axis=0)

array([   0. ,    0. ,  105. ,    6. ,   55. ,    0. ,  232. ,  108. ,
          0. , -172. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
      

In [15]:
from anomaly.base import AnomalyDetector

In [16]:
from river import metrics

In [17]:
from models.rs_hash import RSHash

In [18]:
from river.stream import iter_array

In [38]:
model = RSHash(X.min(axis=0),X.max(axis=0))
ROCAUC = metrics.ROCAUC()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    model = model.learn_one(x_array)           #pas dans le bon sens pour river
    anomaly_score = model.score_one(x_array)   #A FAIRE: il faut que anomaly_score soit entre 1 et 0 pour river
    
    print(y_array,anomaly_score)
    ROCAUC.update(int(y_array[0]),anomaly_score)
print(ROCAUC)

[1] 0.0
[0] 0.0619165880447308
[0] 0.0709969474944409
[0] 0.09957928212115494
[1] 0.1720020767980932
[1] 0.06877338606588995
[0] 0.137897613657609
[0] 0.16206508989178062
[0] 0.18750982498342367
[0] 0.19206038781833953
[1] 0.15972525830252943
[0] 0.23390165976910887
[0] 0.31156488229791046
[0] 0.24126611807050907
[0] 0.25080025069359335
[0] 0.32664405235822996
[0] 0.2813949675683283
[0] 0.2555382501752256
[0] 0.37822213324123166
[0] 0.3144229816029609
[0] 0.3306167486767874
[0] 0.3250500395330594
[0] 0.40621087957659474
[0] 0.3936566401171514
[0] 0.47124266300972956
[0] 0.3284135163125409
[1] 0.31810110331880365
[0] 0.4095008094604396
[0] 0.3616004250020707
[0] 0.30735949465991963
[0] 0.44215454236471674
[0] 0.5292819390908013
[0] 0.4411774722400116
[0] 0.5181131875686188
[1] 0.38380561059648777
[0] 0.6363629087026268
[0] 0.3272688063209632
[0] 0.4362380757372189
[0] 0.4234674394973139
[0] 0.40010942142934086
[0] 0.4787474142855097
[0] 0.5992201881717166
[0] 0.3999109687953583
[0] 0.35

ROCAUC: 28.52%
le modèle ne semble rien apprendre !

In [45]:
from pysad.evaluation import AUROCMetric

In [47]:
from pysad.transform.probability_calibration import ConformalProbabilityCalibrator

In [48]:
model = RSHash(X.min(axis=0),X.max(axis=0))
calibrator = ConformalProbabilityCalibrator(windowed=True, window_size=300)
ROCAUC = AUROCMetric()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    model = model.learn_one(x_array)           #pas dans le bon sens pour river
    anomaly_score = model.score_one(x_array)   #A FAIRE: il faut que anomaly_score soit entre 1 et 0 pour river
    calibrated_score = calibrator.fit_transform(anomaly_score)
#    print(y_array,anomaly_score)
    ROCAUC.update(int(y_array[0]),calibrated_score)
print(ROCAUC.get())

IndexError: tuple index out of range